# Task 4: Recommendation System – CodTech Internship
This notebook builds a **Collaborative Filtering** recommendation model using the `Surprise` library on a sample movie‑ratings dataset. We train an SVD model, evaluate it with RMSE & MAE, and showcase top‑N recommendations for a selected user.

In [ ]:
# 📦 Imports
import pandas as pd
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split
import numpy as np

In [ ]:
# 📊 Create a sample ratings dataset
ratings_dict = {
    'userID': [1,1,1,2,2,3,3,4,4,5],
    'itemID': [101,102,103,102,104,102,103,101,104,103],
    'rating': [5,3,4,2,4,5,3,4,5,4]
}
df = pd.DataFrame(ratings_dict)
df

In [ ]:
# 🔄 Load data into Surprise format
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(df[['userID','itemID','rating']], reader)

In [ ]:
# ✂️ Train–Test Split
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

In [ ]:
# 🧠 Train SVD model
algo = SVD(random_state=42)
algo.fit(trainset)

In [ ]:
# 📈 Evaluate model on test set
predictions = algo.test(testset)
rmse = accuracy.rmse(predictions, verbose=False)
mae = accuracy.mae(predictions, verbose=False)
print(f'🔹 RMSE: {rmse:.4f}')
print(f'🔹 MAE : {mae:.4f}')

In [ ]:
# 🎯 Generate Top‑N recommendations for a user
def get_top_n(predictions, n=3):
    from collections import defaultdict
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

# Run predictions on all (user,item) pairs not in training set
anti_testset = trainset.build_anti_testset()
all_pred = algo.test(anti_testset)
top_n = get_top_n(all_pred, n=2)

# Display recommendations for user 1
user = 1
print(f'Top recommendations for User {user}:')
for iid, est in top_n[user]:
    print(f'  Item {iid} with predicted rating {est:.2f}')

### ✅ Summary
- Built an SVD-based collaborative filtering model using the Surprise library.
- Evaluated performance on a test split with **RMSE** and **MAE**.
- Generated top‑N item recommendations for each user (example shown for *User 1*).
